In [4]:
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any, Generator


from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

def submit_token(token: str) -> Dict:
    return requests.post(f"{BASE_URL}/submit_token", json={"token": token}).json()

def chat(query: str, thread_id: Optional[str] = None, model: str = "gpt-4", temperature: float = 0.7) -> Generator:
    response = requests.post(f"{BASE_URL}/chat", headers=headers, json={
        "query": query, "thread_id": thread_id, "model": model, "temperature": temperature
    }, stream=True)
    return (json.loads(line.decode('utf-8').replace('data: ', '')) for line in response.iter_lines() if line)

def global_search(query: str, limit: int = 200, min_score: float = 0.4) -> Dict:
    return requests.post(f"{BASE_URL}/search/global", headers=headers, 
        json={"query": query, "limit": limit, "min_score": min_score}).json()

def search_transcripts(query: str, meeting_ids: Optional[List[str]] = None, min_score: float = 0.8) -> Dict:
    return requests.post(f"{BASE_URL}/search/transcripts", headers=headers,
        json={"query": query, "meeting_ids": meeting_ids, "min_score": min_score}).json()

def get_meetings(offset: int = 0, limit: int = 50) -> Dict:
    return requests.get(f"{BASE_URL}/meetings/all", headers=headers, 
        params={"offset": offset, "limit": limit}).json()

def get_meeting_details(meeting_id: str) -> Dict:
    return requests.get(f"{BASE_URL}/meeting/{meeting_id}/details", headers=headers).json()

def create_share_link(target_email: str, access_level: str = "READ", 
                     expiration_hours: int = 24, include_existing: bool = True) -> Dict:
    return requests.post(f"{BASE_URL}/share-links", headers=headers, json={
        "access_level": access_level, "target_email": target_email,
        "expiration_hours": expiration_hours, "include_existing_meetings": include_existing
    }).json()

def accept_share_link(token: str, accepting_email: str) -> Dict:
    return requests.post(f"{BASE_URL}/share-links/accept", headers=headers,
        json={"token": token, "accepting_email": accepting_email}).json()

def start_indexing(num_meetings: int = 200) -> Dict:
    return requests.post(f"{BASE_URL}/start_indexing", headers=headers,
        json={"num_meetings": num_meetings}).json()

def get_indexing_status() -> Dict:
    return requests.get(f"{BASE_URL}/indexing_status", headers=headers).json()


Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [5]:
get_meetings()

{'total': 0, 'meetings': []}

In [28]:
from psql_helpers import get_session
from psql_models import Meeting, UserMeeting, Speaker, DiscussionPoint, AccessLevel,User
from sqlalchemy import select, and_, func,distinct

In [10]:
limit=None
offset=0


In [13]:
user_id = vexa.user_id

In [14]:
async with get_session() as session:
    # Build base query
    query = (
        select(
            Meeting.meeting_id,
            Meeting.meeting_name,
            Meeting.timestamp,
            func.array_agg(distinct(Speaker.name)).label('speakers')
        )
        .join(UserMeeting, Meeting.meeting_id == UserMeeting.meeting_id)
        .join(DiscussionPoint, Meeting.meeting_id == DiscussionPoint.meeting_id)
        .join(Speaker, DiscussionPoint.speaker_id == Speaker.id)
        .where(
            and_(
                UserMeeting.user_id == user_id,
                UserMeeting.access_level != AccessLevel.REMOVED.value
            )
        )
        .group_by(Meeting.meeting_id, Meeting.meeting_name, Meeting.timestamp)
        .order_by(Meeting.timestamp.desc())
    )
    
    # Add limit and offset if provided
    if limit is not None:
        query = query.limit(limit)
    if offset:
        query = query.offset(offset)

    # Execute query
    result = await session.execute(query)
    meetings = result.all()

In [15]:
meetings


[]

In [29]:
from psql_helpers import read_table_async

In [30]:
user_id

'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [33]:
q = (UserMeeting.join(User))

AttributeError: type object 'UserMeeting' has no attribute 'join'

In [20]:
await read_table_async(UserMeeting)

,id,meeting_id,user_id,access_level,created_at,created_by,is_owner
0,1,842afeb6-ab24-4a2c-87f2-49a7a1456e12,None,owner,2024-11-06 08:17:02.812937+00:00,None,True
